# Task
Develop a command-line Library Management System in Python with SQLite as the database, including functionalities for managing books (CRUD), students (CRUD and basic login), issuing and returning books, and generating overdue reminders.

## Initialize Database

### Subtask:
Set up the SQLite database and create necessary tables for `Books`, `Students`, and `BorrowedBooks` with appropriate fields (e.g., title, author, student_id, due_date).


**Reasoning**:
To initialize the database and create the required tables, I will use the `sqlite3` module to connect to `library.db` and execute SQL commands for table creation with appropriate columns, primary keys, and foreign key constraints.



In [ ]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Create Books table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    isbn TEXT UNIQUE NOT NULL,
    quantity INTEGER NOT NULL DEFAULT 1
);
''')

# Create Students table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    student_id_number TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL
);
''')

# Create BorrowedBooks table
cursor.execute('''
CREATE TABLE IF NOT EXISTS BorrowedBooks (
    borrow_id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER NOT NULL,
    student_id INTEGER NOT NULL,
    borrow_date TEXT NOT NULL,
    due_date TEXT NOT NULL,
    return_date TEXT,
    FOREIGN KEY (book_id) REFERENCES Books(book_id),
    FOREIGN KEY (student_id) REFERENCES Students(student_id)
);
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database 'library.db' initialized and tables created successfully.")

Database 'library.db' initialized and tables created successfully.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Implement Book CRUD Operations

### Subtask:
Develop functions to Create, Read, Update, and Delete book records in the database.


**Reasoning**:
I will define functions for Create, Read, Update, and Delete operations for books as described in the instructions. This includes functions to add a new book, retrieve a specific book, list all books, update book details, and delete a book, all interacting with the `Books` table in `library.db`.



In [ ]:
import sqlite3

def add_book(title, author, isbn, quantity):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Books (title, author, isbn, quantity) VALUES (?, ?, ?, ?)",
                       (title, author, isbn, quantity))
        conn.commit()
        print(f"Book '{title}' added successfully.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A book with ISBN '{isbn}' already exists.")
        return False
    finally:
        conn.close()

def get_book(identifier):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT * FROM Books WHERE book_id = ? OR isbn = ?"
    cursor.execute(query, (identifier, identifier))
    book = cursor.fetchone()
    conn.close()
    if book:
        # Assuming columns: book_id, title, author, isbn, quantity
        return {
            "book_id": book[0],
            "title": book[1],
            "author": book[2],
            "isbn": book[3],
            "quantity": book[4]
        }
    return None

def list_all_books():
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Books")
    books = cursor.fetchall()
    conn.close()
    book_list = []
    for book in books:
        book_list.append({
            "book_id": book[0],
            "title": book[1],
            "author": book[2],
            "isbn": book[3],
            "quantity": book[4]
        })
    return book_list

def update_book(book_id, title=None, author=None, isbn=None, quantity=None):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    updates = []
    params = []

    if title is not None:
        updates.append("title = ?")
        params.append(title)
    if author is not None:
        updates.append("author = ?")
        params.append(author)
    if isbn is not None:
        updates.append("isbn = ?")
        params.append(isbn)
    if quantity is not None:
        updates.append("quantity = ?")
        params.append(quantity)

    if not updates:
        print("No update parameters provided.")
        conn.close()
        return False

    query = f"UPDATE Books SET {', '.join(updates)} WHERE book_id = ?"
    params.append(book_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Book with ID {book_id} updated successfully.")
            return True
        else:
            print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot update book. ISBN '{isbn}' might already exist for another book.")
        return False
    finally:
        conn.close()

def delete_book(book_id):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Books WHERE book_id = ?", (book_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Book with ID {book_id} deleted successfully.")
            return True
        else:
            print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError: # This might occur if there are foreign key constraints not handled
        print(f"Error: Cannot delete book with ID {book_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        conn.close()


# --- Test the functions (optional, but good for verification) ---
print("\n--- Testing Book CRUD Operations ---")

# 1. Add books
print("Adding books...")
add_book("The Great Gatsby", "F. Scott Fitzgerald", "978-0743273565", 5)
add_book("1984", "George Orwell", "978-0451524935", 3)
add_book("To Kill a Mockingbird", "Harper Lee", "978-0061120084", 7)
add_book("1984", "George Orwell", "978-0451524935", 1) # Attempt to add duplicate ISBN

# 2. List all books
print("\nListing all books...")
all_books = list_all_books()
for book in all_books:
    print(book)

# 3. Get a specific book
print("\nGetting book by ID 1...")
gatsby_book = get_book(1)
print(gatsby_book)

print("\nGetting book by ISBN 978-0451524935...")
orwell_book = get_book("978-0451524935")
print(orwell_book)

print("\nGetting non-existent book...")
non_existent_book = get_book(99)
print(non_existent_book)

# 4. Update a book
print("\nUpdating book ID 1...")
update_book(1, quantity=6, title="The Great Gatsby (Revised)")
updated_gatsby = get_book(1)
print(updated_gatsby)

print("\nUpdating non-existent book ID 99...")
update_book(99, quantity=10)

# 5. Delete a book
print("\nDeleting book ID 3...")
delete_book(3)

print("\nListing all books after deletion...")
all_books_after_delete = list_all_books()
for book in all_books_after_delete:
    print(book)



--- Testing Book CRUD Operations ---
Adding books...
Error: A book with ISBN '978-0743273565' already exists.
Error: A book with ISBN '978-0451524935' already exists.
Book 'To Kill a Mockingbird' added successfully.
Error: A book with ISBN '978-0451524935' already exists.

Listing all books...
{'book_id': 1, 'title': 'The Great Gatsby (Revised)', 'author': 'F. Scott Fitzgerald', 'isbn': '978-0743273565', 'quantity': 6}
{'book_id': 2, 'title': '1984', 'author': 'George Orwell', 'isbn': '978-0451524935', 'quantity': 3}
{'book_id': 4, 'title': 'The Hobbit', 'author': 'J.R.R. Tolkien', 'isbn': '978-0345339683', 'quantity': 2}
{'book_id': 5, 'title': 'Dune', 'author': 'Frank Herbert', 'isbn': '978-0441013593', 'quantity': 0}
{'book_id': 6, 'title': 'To Kill a Mockingbird', 'author': 'Harper Lee', 'isbn': '978-0061120084', 'quantity': 7}

Getting book by ID 1...
{'book_id': 1, 'title': 'The Great Gatsby (Revised)', 'author': 'F. Scott Fitzgerald', 'isbn': '978-0743273565', 'quantity': 6}

G

**Reasoning**:
The previous code failed due to a `SyntaxError` on line 38: `cursor = conn.cursor()l`. There was an extraneous 'l' character after `conn.cursor()`. I will correct this syntax error by removing the 'l'.



In [ ]:
import sqlite3

def add_book(title, author, isbn, quantity):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Books (title, author, isbn, quantity) VALUES (?, ?, ?, ?)",
                       (title, author, isbn, quantity))
        conn.commit()
        print(f"Book '{title}' added successfully.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A book with ISBN '{isbn}' already exists.")
        return False
    finally:
        conn.close()

def get_book(identifier):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT * FROM Books WHERE book_id = ? OR isbn = ?"
    cursor.execute(query, (identifier, identifier))
    book = cursor.fetchone()
    conn.close()
    if book:
        # Assuming columns: book_id, title, author, isbn, quantity
        return {
            "book_id": book[0],
            "title": book[1],
            "author": book[2],
            "isbn": book[3],
            "quantity": book[4]
        }
    return None

def list_all_books():
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Books")
    books = cursor.fetchall()
    conn.close()
    book_list = []
    for book in books:
        book_list.append({
            "book_id": book[0],
            "title": book[1],
            "author": book[2],
            "isbn": book[3],
            "quantity": book[4]
        })
    return book_list

def update_book(book_id, title=None, author=None, isbn=None, quantity=None):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    updates = []
    params = []

    if title is not None:
        updates.append("title = ?")
        params.append(title)
    if author is not None:
        updates.append("author = ?")
        params.append(author)
    if isbn is not None:
        updates.append("isbn = ?")
        params.append(isbn)
    if quantity is not None:
        updates.append("quantity = ?")
        params.append(quantity)

    if not updates:
        print("No update parameters provided.")
        conn.close()
        return False

    query = f"UPDATE Books SET {', '.join(updates)} WHERE book_id = ?"
    params.append(book_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Book with ID {book_id} updated successfully.")
            return True
        else:
            print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot update book. ISBN '{isbn}' might already exist for another book.")
        return False
    finally:
        conn.close()

def delete_book(book_id):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Books WHERE book_id = ?", (book_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Book with ID {book_id} deleted successfully.")
            return True
        else:
            print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError: # This might occur if there are foreign key constraints not handled
        print(f"Error: Cannot delete book with ID {book_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        conn.close()


# --- Test the functions (optional, but good for verification) ---
print("\n--- Testing Book CRUD Operations ---")

# 1. Add books
print("Adding books...")
add_book("The Great Gatsby", "F. Scott Fitzgerald", "978-0743273565", 5)
add_book("1984", "George Orwell", "978-0451524935", 3)
add_book("To Kill a Mockingbird", "Harper Lee", "978-0061120084", 7)
add_book("1984", "George Orwell", "978-0451524935", 1) # Attempt to add duplicate ISBN

# 2. List all books
print("\nListing all books...")
all_books = list_all_books()
for book in all_books:
    print(book)

# 3. Get a specific book
print("\nGetting book by ID 1...")
gatsby_book = get_book(1)
print(gatsby_book)

print("\nGetting book by ISBN 978-0451524935...")
orwell_book = get_book("978-0451524935")
print(orwell_book)

print("\nGetting non-existent book...")
non_existent_book = get_book(99)
print(non_existent_book)

# 4. Update a book
print("\nUpdating book ID 1...")
update_book(1, quantity=6, title="The Great Gatsby (Revised)")
updated_gatsby = get_book(1)
print(updated_gatsby)

print("\nUpdating non-existent book ID 99...")
update_book(99, quantity=10)

# 5. Delete a book
print("\nDeleting book ID 3...")
delete_book(3)

print("\nListing all books after deletion...")
all_books_after_delete = list_all_books()
for book in all_books_after_delete:
    print(book)


--- Testing Book CRUD Operations ---
Adding books...
Book 'The Great Gatsby' added successfully.
Book '1984' added successfully.
Book 'To Kill a Mockingbird' added successfully.
Error: A book with ISBN '978-0451524935' already exists.

Listing all books...
{'book_id': 1, 'title': 'The Great Gatsby', 'author': 'F. Scott Fitzgerald', 'isbn': '978-0743273565', 'quantity': 5}
{'book_id': 2, 'title': '1984', 'author': 'George Orwell', 'isbn': '978-0451524935', 'quantity': 3}
{'book_id': 3, 'title': 'To Kill a Mockingbird', 'author': 'Harper Lee', 'isbn': '978-0061120084', 'quantity': 7}

Getting book by ID 1...
{'book_id': 1, 'title': 'The Great Gatsby', 'author': 'F. Scott Fitzgerald', 'isbn': '978-0743273565', 'quantity': 5}

Getting book by ISBN 978-0451524935...
{'book_id': 2, 'title': '1984', 'author': 'George Orwell', 'isbn': '978-0451524935', 'quantity': 3}

Getting non-existent book...
None

Updating book ID 1...
Book with ID 1 updated successfully.
{'book_id': 1, 'title': 'The Gre

## Implement Student CRUD Operations

### Subtask:
Develop functions to Create, Read, Update, and Delete student records, including a basic student login functionality.


**Reasoning**:
I will define functions for Create, Read, Update, Delete, and Login operations for students as described in the instructions. This includes functions to add a new student, retrieve a specific student, list all students, update student details, delete a student, and verify login credentials, all interacting with the `Students` table in `library.db`.

**Reasoning**:
I will implement the `add_student`, `get_student`, `list_all_students`, `update_student`, `delete_student`, and `login_student` functions as specified, including error handling and database interactions, and then provide test cases to demonstrate their functionality.



In [ ]:
import sqlite3

def add_student(name, student_id_number, password):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Students (name, student_id_number, password) VALUES (?, ?, ?)",
                       (name, student_id_number, password))
        conn.commit()
        print(f"Student '{name}' added successfully with ID: {cursor.lastrowid}.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A student with ID number '{student_id_number}' already exists.")
        return False
    finally:
        conn.close()

def get_student(identifier):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id = ? OR student_id_number = ?"
    cursor.execute(query, (identifier, identifier))
    student = cursor.fetchone()
    conn.close()
    if student:
        return {
            "student_id": student[0],
            "name": student[1],
            "student_id_number": student[2]
        }
    return None

def list_all_students():
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    cursor.execute("SELECT student_id, name, student_id_number FROM Students")
    students = cursor.fetchall()
    conn.close()
    student_list = []
    for student in students:
        student_list.append({
            "student_id": student[0],
            "name": student[1],
            "student_id_number": student[2]
        })
    return student_list

def update_student(student_id, name=None, student_id_number=None, password=None):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    updates = []
    params = []

    if name is not None:
        updates.append("name = ?")
        params.append(name)
    if student_id_number is not None:
        updates.append("student_id_number = ?")
        params.append(student_id_number)
    if password is not None:
        updates.append("password = ?")
        params.append(password)

    if not updates:
        print("No update parameters provided.")
        conn.close()
        return False

    query = f"UPDATE Students SET {', '.join(updates)} WHERE student_id = ?"
    params.append(student_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} updated successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot update student. Student ID number '{student_id_number}' might already exist for another student.")
        return False
    finally:
        conn.close()

def delete_student(student_id):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Students WHERE student_id = ?", (student_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} deleted successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot delete student with ID {student_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        conn.close()

def login_student(student_id_number, password):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id_number = ? AND password = ?"
    cursor.execute(query, (student_id_number, password))
    student = cursor.fetchone()
    conn.close()
    if student:
        print(f"Login successful for {student[1]}.")
        return {"student_id": student[0], "name": student[1], "student_id_number": student[2]}
    else:
        print("Invalid student ID number or password.")
        return None

# --- Test the functions ---
print("\n--- Testing Student CRUD Operations and Login ---")

# 1. Add students
print("Adding students...")
add_student("Alice Smith", "S001", "pass123")
add_student("Bob Johnson", "S002", "secret")
add_student("Charlie Brown", "S003", "charliepwd")
add_student("Alice Smith", "S001", "anotherpass") # Attempt to add duplicate student_id_number

# 2. List all students
print("\nListing all students...")
all_students = list_all_students()
for student in all_students:
    print(student)

# 3. Get a specific student
print("\nGetting student by ID 1...")
alice = get_student(1)
print(alice)

print("\nGetting student by ID number S002...")
bob = get_student("S002")
print(bob)

print("\nGetting non-existent student...")
non_existent_student = get_student(99)
print(non_existent_student)

# 4. Update a student
print("\nUpdating student ID 1...")
update_student(1, name="Alice Williams", password="newpass")
updated_alice = get_student(1)
print(updated_alice)

print("\nUpdating non-existent student ID 99...")
update_student(99, name="Non Existent")

# 5. Delete a student
print("\nDeleting student ID 3...")
delete_student(3)

print("\nListing all students after deletion...")
all_students_after_delete = list_all_students()
for student in all_students_after_delete:
    print(student)

# 6. Test login
print("\nTesting student login...")
logged_in_alice = login_student("S001", "newpass")
print(f"Logged in user: {logged_in_alice}")

failed_login = login_student("S002", "wrongpass")
print(f"Failed login attempt: {failed_login}")


--- Testing Student CRUD Operations and Login ---
Adding students...
Error: A student with ID number 'S001' already exists.
Error: A student with ID number 'S002' already exists.
Student 'Charlie Brown' added successfully with ID: 6.
Error: A student with ID number 'S001' already exists.

Listing all students...
{'student_id': 1, 'name': 'Alice Williams', 'student_id_number': 'S001'}
{'student_id': 2, 'name': 'Bob Johnson', 'student_id_number': 'S002'}
{'student_id': 4, 'name': 'Student One', 'student_id_number': 'S004'}
{'student_id': 5, 'name': 'Student Two', 'student_id_number': 'S005'}
{'student_id': 6, 'name': 'Charlie Brown', 'student_id_number': 'S003'}

Getting student by ID 1...
{'student_id': 1, 'name': 'Alice Williams', 'student_id_number': 'S001'}

Getting student by ID number S002...
{'student_id': 2, 'name': 'Bob Johnson', 'student_id_number': 'S002'}

Getting non-existent student...
None

Updating student ID 1...
Student with ID 1 updated successfully.
{'student_id': 1,

**Reasoning**:
The previous code failed due to a `SyntaxError` on line 89: `cursor = conn.cursor()l`. There was an extraneous 'l' character after `conn.cursor()`. I will correct this syntax error by removing the 'l'.



In [ ]:
import sqlite3

def add_student(name, student_id_number, password):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Students (name, student_id_number, password) VALUES (?, ?, ?)",
                       (name, student_id_number, password))
        conn.commit()
        print(f"Student '{name}' added successfully with ID: {cursor.lastrowid}.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A student with ID number '{student_id_number}' already exists.")
        return False
    finally:
        conn.close()

def get_student(identifier):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id = ? OR student_id_number = ?"
    cursor.execute(query, (identifier, identifier))
    student = cursor.fetchone()
    conn.close()
    if student:
        return {
            "student_id": student[0],
            "name": student[1],
            "student_id_number": student[2]
        }
    return None

def list_all_students():
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    cursor.execute("SELECT student_id, name, student_id_number FROM Students")
    students = cursor.fetchall()
    conn.close()
    student_list = []
    for student in students:
        student_list.append({
            "student_id": student[0],
            "name": student[1],
            "student_id_number": student[2]
        })
    return student_list

def update_student(student_id, name=None, student_id_number=None, password=None):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    updates = []
    params = []

    if name is not None:
        updates.append("name = ?")
        params.append(name)
    if student_id_number is not None:
        updates.append("student_id_number = ?")
        params.append(student_id_number)
    if password is not None:
        updates.append("password = ?")
        params.append(password)

    if not updates:
        print("No update parameters provided.")
        conn.close()
        return False

    query = f"UPDATE Students SET {', '.join(updates)} WHERE student_id = ?"
    params.append(student_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} updated successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot update student. Student ID number '{student_id_number}' might already exist for another student.")
        return False
    finally:
        conn.close()

def delete_student(student_id):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Students WHERE student_id = ?", (student_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} deleted successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot delete student with ID {student_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        conn.close()

def login_student(student_id_number, password):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id_number = ? AND password = ?"
    cursor.execute(query, (student_id_number, password))
    student = cursor.fetchone()
    conn.close()
    if student:
        print(f"Login successful for {student[1]}.")
        return {"student_id": student[0], "name": student[1], "student_id_number": student[2]}
    else:
        print("Invalid student ID number or password.")
        return None

# --- Test the functions ---
print("\n--- Testing Student CRUD Operations and Login ---")

# 1. Add students
print("Adding students...")
add_student("Alice Smith", "S001", "pass123")
add_student("Bob Johnson", "S002", "secret")
add_student("Charlie Brown", "S003", "charliepwd")
add_student("Alice Smith", "S001", "anotherpass") # Attempt to add duplicate student_id_number

# 2. List all students
print("\nListing all students...")
all_students = list_all_students()
for student in all_students:
    print(student)

# 3. Get a specific student
print("\nGetting student by ID 1...")
alice = get_student(1)
print(alice)

print("\nGetting student by ID number S002...")
bob = get_student("S002")
print(bob)

print("\nGetting non-existent student...")
non_existent_student = get_student(99)
print(non_existent_student)

# 4. Update a student
print("\nUpdating student ID 1...")
update_student(1, name="Alice Williams", password="newpass")
updated_alice = get_student(1)
print(updated_alice)

print("\nUpdating non-existent student ID 99...")
update_student(99, name="Non Existent")

# 5. Delete a student
print("\nDeleting student ID 3...")
delete_student(3)

print("\nListing all students after deletion...")
all_students_after_delete = list_all_students()
for student in all_students_after_delete:
    print(student)

# 6. Test login
print("\nTesting student login...")
logged_in_alice = login_student("S001", "newpass")
print(f"Logged in user: {logged_in_alice}")

failed_login = login_student("S002", "wrongpass")
print(f"Failed login attempt: {failed_login}")


--- Testing Student CRUD Operations and Login ---
Adding students...
Student 'Alice Smith' added successfully with ID: 1.
Student 'Bob Johnson' added successfully with ID: 2.
Student 'Charlie Brown' added successfully with ID: 3.
Error: A student with ID number 'S001' already exists.

Listing all students...
{'student_id': 1, 'name': 'Alice Smith', 'student_id_number': 'S001'}
{'student_id': 2, 'name': 'Bob Johnson', 'student_id_number': 'S002'}
{'student_id': 3, 'name': 'Charlie Brown', 'student_id_number': 'S003'}

Getting student by ID 1...
{'student_id': 1, 'name': 'Alice Smith', 'student_id_number': 'S001'}

Getting student by ID number S002...
{'student_id': 2, 'name': 'Bob Johnson', 'student_id_number': 'S002'}

Getting non-existent student...
None

Updating student ID 1...
Student with ID 1 updated successfully.
{'student_id': 1, 'name': 'Alice Williams', 'student_id_number': 'S001'}

Updating non-existent student ID 99...
No student found with ID 99.

Deleting student ID 3...

## Manage Book Issuance and Return

### Subtask:
Create functions to handle the process of issuing a book to a student and marking a book as returned, updating the `BorrowedBooks` table accordingly.


**Reasoning**:
I will define the `issue_book` and `return_book` functions to manage the book issuance and return process, including checks for book availability, updating quantities, and handling borrowed records. Then, I will add comprehensive test cases to verify their functionality.



In [ ]:
import sqlite3
import datetime

# Re-using existing CRUD functions for Books and Students for context and testing
# (Assuming these functions are already defined in the environment)
# from previous cells: add_book, get_book, update_book, add_student, get_student

def add_book(title, author, isbn, quantity):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Books (title, author, isbn, quantity) VALUES (?, ?, ?, ?)",
                       (title, author, isbn, quantity))
        conn.commit()
        print(f"Book '{title}' added successfully.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A book with ISBN '{isbn}' already exists.")
        return False
    finally:
        conn.close()

def get_book(identifier):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT * FROM Books WHERE book_id = ? OR isbn = ?"
    cursor.execute(query, (identifier, identifier))
    book = cursor.fetchone()
    conn.close()
    if book:
        return {
            "book_id": book[0],
            "title": book[1],
            "author": book[2],
            "isbn": book[3],
            "quantity": book[4]
        }
    return None

def update_book(book_id, title=None, author=None, isbn=None, quantity=None):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    updates = []
    params = []

    if title is not None:
        updates.append("title = ?")
        params.append(title)
    if author is not None:
        updates.append("author = ?")
        params.append(author)
    if isbn is not None:
        updates.append("isbn = ?")
        params.append(isbn)
    if quantity is not None:
        updates.append("quantity = ?")
        params.append(quantity)

    if not updates:
        print("No update parameters provided.")
        conn.close()
        return False

    query = f"UPDATE Books SET {', '.join(updates)} WHERE book_id = ?"
    params.append(book_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            # print(f"Book with ID {book_id} updated successfully.")
            return True
        else:
            # print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError:
        # print(f"Error: Cannot update book. ISBN '{isbn}' might already exist for another book.")
        return False
    finally:
        conn.close()

def add_student(name, student_id_number, password):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Students (name, student_id_number, password) VALUES (?, ?, ?)",
                       (name, student_id_number, password))
        conn.commit()
        print(f"Student '{name}' added successfully with ID: {cursor.lastrowid}.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A student with ID number '{student_id_number}' already exists.")
        return False
    finally:
        conn.close()

def get_student(identifier):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id = ? OR student_id_number = ?"
    cursor.execute(query, (identifier, identifier))
    student = cursor.fetchone()
    conn.close()
    if student:
        return {
            "student_id": student[0],
            "name": student[1],
            "student_id_number": student[2]
        }
    return None

# 1. Define issue_book function
def issue_book(book_id, student_id, borrow_date, due_date):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        # Check if book exists and is available
        book = get_book(book_id)
        if not book:
            print(f"Error: Book with ID {book_id} not found.")
            return False
        if book['quantity'] <= 0:
            print(f"Error: Book '{book['title']}' (ID: {book_id}) is currently out of stock.")
            return False

        # Check if student exists
        student = get_student(student_id)
        if not student:
            print(f"Error: Student with ID {student_id} not found.")
            return False

        # Insert into BorrowedBooks
        cursor.execute(
            "INSERT INTO BorrowedBooks (book_id, student_id, borrow_date, due_date) VALUES (?, ?, ?, ?)",
            (book_id, student_id, borrow_date, due_date)
        )
        borrow_id = cursor.lastrowid

        # Decrement book quantity
        update_book(book_id, quantity=book['quantity'] - 1)

        conn.commit()
        print(f"Book '{book['title']}' issued to '{student['name']}' (Borrow ID: {borrow_id}) successfully.")
        return borrow_id
    except Exception as e:
        conn.rollback()
        print(f"Error issuing book: {e}")
        return False
    finally:
        conn.close()

# 2. Define return_book function
def return_book(borrow_id, return_date):
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    try:
        # Check if borrow record exists and is not already returned
        cursor.execute("SELECT book_id, return_date FROM BorrowedBooks WHERE borrow_id = ?", (borrow_id,))
        borrow_record = cursor.fetchone()

        if not borrow_record:
            print(f"Error: Borrow record with ID {borrow_id} not found.")
            return False
        if borrow_record[1] is not None: # return_date column
            print(f"Error: Book from borrow record ID {borrow_id} has already been returned on {borrow_record[1]}.")
            return False

        book_id = borrow_record[0]

        # Update return_date in BorrowedBooks
        cursor.execute("UPDATE BorrowedBooks SET return_date = ? WHERE borrow_id = ?", (return_date, borrow_id))

        # Increment book quantity
        book = get_book(book_id)
        if book:
            update_book(book_id, quantity=book['quantity'] + 1)
        else:
            print(f"Warning: Book with ID {book_id} not found for incrementing quantity after return.")

        conn.commit()
        print(f"Book from borrow ID {borrow_id} returned successfully.")
        return True
    except Exception as e:
        conn.rollback()
        print(f"Error returning book: {e}")
        return False
    finally:
        conn.close()


# --- Test Cases ---
print("\n--- Testing Book Issuance and Return Operations ---")

# Ensure there are students and books for testing
# Add students if not already present from previous cell tests
add_student("Student One", "S004", "pass4")
add_student("Student Two", "S005", "pass5")

# Add books if not already present or reset quantities for fresh testing
# For simplicity, we'll just add new ones or rely on previous ones if they exist
add_book("The Hobbit", "J.R.R. Tolkien", "978-0345339683", 2) # Book ID expected to be 3 or 4 based on previous deletions
add_book("Dune", "Frank Herbert", "978-0441013593", 1)

# Retrieve current book and student IDs for testing
book_hobbit = get_book("978-0345339683")
book_dune = get_book("978-0441013593")
student_one = get_student("S004")
student_two = get_student("S005")

hobbit_id = book_hobbit['book_id'] if book_hobbit else None
dune_id = book_dune['book_id'] if book_dune else None
student_one_id = student_one['student_id'] if student_one else None
student_two_id = student_two['student_id'] if student_two else None

print(f"\nInitial quantities: Hobbit (ID: {hobbit_id}) - {get_book(hobbit_id)['quantity']}, Dune (ID: {dune_id}) - {get_book(dune_id)['quantity']}")

# 3.b. Successfully issue a book
print("\nIssuing 'The Hobbit' to Student One...")
borrow_id_hobbit = issue_book(hobbit_id, student_one_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=7)))

print("\nIssuing 'Dune' to Student Two...")
borrow_id_dune = issue_book(dune_id, student_two_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=14)))

print(f"\nQuantities after issuance: Hobbit (ID: {hobbit_id}) - {get_book(hobbit_id)['quantity']}, Dune (ID: {dune_id}) - {get_book(dune_id)['quantity']}")

# 3.c. Attempt to issue a book that is out of stock (Dune has quantity 0 now)
print("\nAttempting to issue 'Dune' again (should be out of stock)...")
issue_book(dune_id, student_one_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=7)))

# Attempt to issue a non-existent book
print("\nAttempting to issue a non-existent book (ID: 999)...")
issue_book(999, student_one_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=7)))

# 3.d. Successfully return a book
print("\nReturning 'The Hobbit' (Borrow ID: {borrow_id_hobbit})...")
return_book(borrow_id_hobbit, str(datetime.date.today() + datetime.timedelta(days=5)))

print(f"\nQuantities after return: Hobbit (ID: {hobbit_id}) - {get_book(hobbit_id)['quantity']}, Dune (ID: {dune_id}) - {get_book(dune_id)['quantity']}")

# 3.e. Attempt to return a book that was not borrowed or already returned
print("\nAttempting to return an already returned book (Borrow ID: {borrow_id_hobbit})...")
return_book(borrow_id_hobbit, str(datetime.date.today() + datetime.timedelta(days=6)))

print("\nAttempting to return a non-existent borrow record (ID: 999)...")
return_book(999, str(datetime.date.today() + datetime.timedelta(days=10)))

# Verify BorrowedBooks entries
print("\nVerifying BorrowedBooks table entries:")
conn = sqlite3.connect('library.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM BorrowedBooks")
borrowed_records = cursor.fetchall()
conn.close()
for record in borrowed_records:
    print(record)



--- Testing Book Issuance and Return Operations ---
Student 'Student One' added successfully with ID: 4.
Student 'Student Two' added successfully with ID: 5.
Book 'The Hobbit' added successfully.
Book 'Dune' added successfully.

Initial quantities: Hobbit (ID: 4) - 2, Dune (ID: 5) - 1

Issuing 'The Hobbit' to Student One...
Error issuing book: database is locked

Issuing 'Dune' to Student Two...
Error issuing book: database is locked

Quantities after issuance: Hobbit (ID: 4) - 2, Dune (ID: 5) - 1

Attempting to issue 'Dune' again (should be out of stock)...
Error issuing book: database is locked

Attempting to issue a non-existent book (ID: 999)...
Error: Book with ID 999 not found.

Returning 'The Hobbit' (Borrow ID: {borrow_id_hobbit})...
Error: Borrow record with ID False not found.

Quantities after return: Hobbit (ID: 4) - 2, Dune (ID: 5) - 1

Attempting to return an already returned book (Borrow ID: {borrow_id_hobbit})...
Error: Borrow record with ID False not found.

Attemptin

**Reasoning**:
The previous code failed because of 'database is locked' errors due to multiple, uncoordinated database connections being opened and closed within nested function calls. To fix this, I will refactor the helper functions (`get_book`, `update_book`, `get_student`) to accept an existing `connection` and `cursor` object, allowing the `issue_book` and `return_book` functions to manage a single connection and transaction for all their internal operations, preventing concurrent access issues. I will also ensure proper error handling and variable usage in the test cases.



In [ ]:
import sqlite3
import datetime

# Helper function to get a database connection and cursor
def get_db_connection():
    conn = sqlite3.connect('library.db')
    conn.row_factory = sqlite3.Row # Allows accessing columns by name
    return conn

# Refactored CRUD functions for Books and Students to accept optional connection/cursor
def add_book(title, author, isbn, quantity, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Books (title, author, isbn, quantity) VALUES (?, ?, ?, ?)",
                       (title, author, isbn, quantity))
        conn.commit()
        print(f"Book '{title}' added successfully.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A book with ISBN '{isbn}' already exists.")
        return False
    finally:
        if close_conn: conn.close()

def get_book(identifier, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT * FROM Books WHERE book_id = ? OR isbn = ?"
    cursor.execute(query, (identifier, identifier))
    book = cursor.fetchone()
    if close_conn: conn.close()
    if book:
        return {
            "book_id": book['book_id'],
            "title": book['title'],
            "author": book['author'],
            "isbn": book['isbn'],
            "quantity": book['quantity']
        }
    return None

def update_book(book_id, title=None, author=None, isbn=None, quantity=None, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    updates = []
    params = []

    if title is not None:
        updates.append("title = ?")
        params.append(title)
    if author is not None:
        updates.append("author = ?")
        params.append(author)
    if isbn is not None:
        updates.append("isbn = ?")
        params.append(isbn)
    if quantity is not None:
        updates.append("quantity = ?")
        params.append(quantity)

    if not updates:
        # print("No update parameters provided.") # Suppress this print for internal updates
        if close_conn: conn.close()
        return False

    query = f"UPDATE Books SET {', '.join(updates)} WHERE book_id = ?"
    params.append(book_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit() # Commit here for this specific update
        if cursor.rowcount > 0:
            # print(f"Book with ID {book_id} updated successfully.")
            return True
        else:
            # print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError:
        # print(f"Error: Cannot update book. ISBN '{isbn}' might already exist for another book.")
        return False
    finally:
        if close_conn: conn.close()

def add_student(name, student_id_number, password, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Students (name, student_id_number, password) VALUES (?, ?, ?)",
                       (name, student_id_number, password))
        conn.commit()
        print(f"Student '{name}' added successfully with ID: {cursor.lastrowid}.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A student with ID number '{student_id_number}' already exists.")
        return False
    finally:
        if close_conn: conn.close()

def get_student(identifier, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id = ? OR student_id_number = ?"
    cursor.execute(query, (identifier, identifier))
    student = cursor.fetchone()
    if close_conn: conn.close()
    if student:
        return {
            "student_id": student['student_id'],
            "name": student['name'],
            "student_id_number": student['student_id_number']
        }
    return None

# 1. Define issue_book function
def issue_book(book_id, student_id, borrow_date, due_date):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        # Check if book exists and is available
        book = get_book(book_id, conn) # Pass connection to helper
        if not book:
            print(f"Error: Book with ID {book_id} not found.")
            return False
        if book['quantity'] <= 0:
            print(f"Error: Book '{book['title']}' (ID: {book_id}) is currently out of stock.")
            return False

        # Check if student exists
        student = get_student(student_id, conn) # Pass connection to helper
        if not student:
            print(f"Error: Student with ID {student_id} not found.")
            return False

        # Insert into BorrowedBooks
        cursor.execute(
            "INSERT INTO BorrowedBooks (book_id, student_id, borrow_date, due_date) VALUES (?, ?, ?, ?)",
            (book_id, student_id, borrow_date, due_date)
        )
        borrow_id = cursor.lastrowid

        # Decrement book quantity
        update_book(book_id, quantity=book['quantity'] - 1, conn=conn) # Pass connection to helper

        conn.commit()
        print(f"Book '{book['title']}' issued to '{student['name']}' (Borrow ID: {borrow_id}) successfully.")
        return borrow_id
    except Exception as e:
        conn.rollback()
        print(f"Error issuing book: {e}")
        return False
    finally:
        conn.close()

# 2. Define return_book function
def return_book(borrow_id, return_date):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        # Check if borrow record exists and is not already returned
        cursor.execute("SELECT book_id, return_date FROM BorrowedBooks WHERE borrow_id = ?", (borrow_id,))
        borrow_record = cursor.fetchone()

        if not borrow_record:
            print(f"Error: Borrow record with ID {borrow_id} not found.")
            return False
        if borrow_record['return_date'] is not None:
            print(f"Error: Book from borrow record ID {borrow_id} has already been returned on {borrow_record['return_date']}.")
            return False

        book_id = borrow_record['book_id']

        # Update return_date in BorrowedBooks
        cursor.execute("UPDATE BorrowedBooks SET return_date = ? WHERE borrow_id = ?", (return_date, borrow_id))

        # Increment book quantity
        book = get_book(book_id, conn) # Pass connection to helper
        if book:
            update_book(book_id, quantity=book['quantity'] + 1, conn=conn) # Pass connection to helper
        else:
            print(f"Warning: Book with ID {book_id} not found for incrementing quantity after return.")

        conn.commit()
        print(f"Book from borrow ID {borrow_id} returned successfully.")
        return True
    except Exception as e:
        conn.rollback()
        print(f"Error returning book: {e}")
        return False
    finally:
        conn.close()


# --- Test Cases ---
print("\n--- Testing Book Issuance and Return Operations ---")

# Ensure there are students and books for testing
# Add students if not already present from previous cell tests
add_student("Student One", "S004", "pass4")
add_student("Student Two", "S005", "pass5")

# Add books if not already present or reset quantities for fresh testing
# For simplicity, we'll just add new ones or rely on previous ones if they exist
add_book("The Hobbit", "J.R.R. Tolkien", "978-0345339683", 2)
add_book("Dune", "Frank Herbert", "978-0441013593", 1)

# Retrieve current book and student IDs for testing
# Need to re-fetch to ensure the latest state after potential additions/updates from this or previous cells
book_hobbit = get_book("978-0345339683")
book_dune = get_book("978-0441013593")
student_one = get_student("S004")
student_two = get_student("S005")

hobbit_id = book_hobbit['book_id'] if book_hobbit else None
dune_id = book_dune['book_id'] if book_dune else None
student_one_id = student_one['student_id'] if student_one else None
student_two_id = student_two['student_id'] if student_two else None

print(f"\nInitial quantities: Hobbit (ID: {hobbit_id}) - {get_book(hobbit_id)['quantity'] if hobbit_id else 'N/A'}, Dune (ID: {dune_id}) - {get_book(dune_id)['quantity'] if dune_id else 'N/A'}")

# 3.b. Successfully issue a book
print("\nIssuing 'The Hobbit' to Student One...")
borrow_id_hobbit = issue_book(hobbit_id, student_one_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=7)))

print("\nIssuing 'Dune' to Student Two...")
borrow_id_dune = issue_book(dune_id, student_two_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=14)))

# Ensure we have actual IDs for subsequent tests
if borrow_id_hobbit is False: borrow_id_hobbit = -1 # Use a dummy invalid ID
if borrow_id_dune is False: borrow_id_dune = -1

print(f"\nQuantities after issuance: Hobbit (ID: {hobbit_id}) - {get_book(hobbit_id)['quantity'] if hobbit_id else 'N/A'}, Dune (ID: {dune_id}) - {get_book(dune_id)['quantity'] if dune_id else 'N/A'}")

# 3.c. Attempt to issue a book that is out of stock (Dune has quantity 0 now)
print("\nAttempting to issue 'Dune' again (should be out of stock)...")
issue_book(dune_id, student_one_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=7)))

# Attempt to issue a non-existent book
print("\nAttempting to issue a non-existent book (ID: 999)...")
issue_book(999, student_one_id, str(datetime.date.today()), str(datetime.date.today() + datetime.timedelta(days=7)))

# 3.d. Successfully return a book
print(f"\nReturning 'The Hobbit' (Borrow ID: {borrow_id_hobbit})...")
return_book(borrow_id_hobbit, str(datetime.date.today() + datetime.timedelta(days=5)))

print(f"\nQuantities after return: Hobbit (ID: {hobbit_id}) - {get_book(hobbit_id)['quantity'] if hobbit_id else 'N/A'}, Dune (ID: {dune_id}) - {get_book(dune_id)['quantity'] if dune_id else 'N/A'}")

# 3.e. Attempt to return a book that was not borrowed or already returned
print(f"\nAttempting to return an already returned book (Borrow ID: {borrow_id_hobbit})...")
return_book(borrow_id_hobbit, str(datetime.date.today() + datetime.timedelta(days=6)))

print("\nAttempting to return a non-existent borrow record (ID: 999)...")
return_book(999, str(datetime.date.today() + datetime.timedelta(days=10)))

# Verify BorrowedBooks entries
print("\nVerifying BorrowedBooks table entries:")
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute("SELECT * FROM BorrowedBooks")
borrowed_records = cursor.fetchall()
conn.close()
for record in borrowed_records:
    print(dict(record)) # Print as dictionary for better readability


--- Testing Book Issuance and Return Operations ---
Error: A student with ID number 'S004' already exists.
Error: A student with ID number 'S005' already exists.
Error: A book with ISBN '978-0345339683' already exists.
Error: A book with ISBN '978-0441013593' already exists.

Initial quantities: Hobbit (ID: 4) - 2, Dune (ID: 5) - 1

Issuing 'The Hobbit' to Student One...
Book 'The Hobbit' issued to 'Student One' (Borrow ID: 1) successfully.

Issuing 'Dune' to Student Two...
Book 'Dune' issued to 'Student Two' (Borrow ID: 2) successfully.

Quantities after issuance: Hobbit (ID: 4) - 1, Dune (ID: 5) - 0

Attempting to issue 'Dune' again (should be out of stock)...
Error: Book 'Dune' (ID: 5) is currently out of stock.

Attempting to issue a non-existent book (ID: 999)...
Error: Book with ID 999 not found.

Returning 'The Hobbit' (Borrow ID: 1)...
Book from borrow ID 1 returned successfully.

Quantities after return: Hobbit (ID: 4) - 2, Dune (ID: 5) - 0

Attempting to return an already re

## Implement Due-Date Reminders

### Subtask:
Develop a function to identify books that are past their due date and generate reminders. Initially, this can be a simple console output.


**Reasoning**:
I will define the `get_overdue_books` function to identify books that are past their due date, retrieving details from `BorrowedBooks`, `Books`, and `Students` tables. I will then add test cases to demonstrate its functionality for current and historical dates and print the results.



In [ ]:
import sqlite3
import datetime

# Re-using existing helper function from previous cells
def get_db_connection():
    conn = sqlite3.connect('library.db')
    conn.row_factory = sqlite3.Row # Allows accessing columns by name
    return conn

def get_overdue_books(current_date=None):
    conn = get_db_connection()
    cursor = conn.cursor()

    if current_date is None:
        current_date = datetime.date.today()
    elif isinstance(current_date, str):
        current_date = datetime.date.fromisoformat(current_date)

    # Convert current_date to string for SQL comparison
    current_date_str = str(current_date)

    query = """
    SELECT
        bb.borrow_id,
        b.book_id,
        b.title AS book_title,
        b.author AS book_author,
        s.student_id,
        s.name AS student_name,
        s.student_id_number,
        bb.borrow_date,
        bb.due_date
    FROM BorrowedBooks bb
    JOIN Books b ON bb.book_id = b.book_id
    JOIN Students s ON bb.student_id = s.student_id
    WHERE bb.return_date IS NULL AND bb.due_date < ?
    """
    cursor.execute(query, (current_date_str,))
    overdue_records = cursor.fetchall()
    conn.close()

    overdue_list = []
    for record in overdue_records:
        overdue_list.append(dict(record))
    return overdue_list

# --- Test Cases for Due-Date Reminders ---
print("\n--- Testing Due-Date Reminders ---")

# Ensure some books are borrowed and overdue for testing purposes
# (Assuming previous issue_book calls have set up some borrowed records)
# Let's ensure the 'Dune' book is still borrowed and possibly overdue
# If we want a fresh overdue scenario, we can re-issue or ensure a specific due date

# To ensure a clean test for overdue, let's update a borrowed book's due date to the past.
conn = get_db_connection()
cursor = conn.cursor()

# Find an existing borrowed book that hasn't been returned
cursor.execute("SELECT borrow_id, book_id, student_id, borrow_date FROM BorrowedBooks WHERE return_date IS NULL LIMIT 1")
active_borrow = cursor.fetchone()

if active_borrow:
    # Update its due_date to a past date to make it overdue
    past_due_date = str(datetime.date.today() - datetime.timedelta(days=10))
    cursor.execute("UPDATE BorrowedBooks SET due_date = ? WHERE borrow_id = ?", (past_due_date, active_borrow['borrow_id']))
    conn.commit()
    print(f"Updated borrow_id {active_borrow['borrow_id']} to have due_date: {past_due_date} for testing.")
else:
    print("No active borrowed books found to set as overdue. Please ensure some books are issued.")

conn.close()

# a. Call get_overdue_books without arguments (for today's date)
print("\n--- Overdue books as of today ---")
overdue_today = get_overdue_books()
if overdue_today:
    for book in overdue_today:
        print(f"REMINDER: Book '{book['book_title']}' (ID: {book['book_id']}) by {book['book_author']} is overdue. "
              f"Borrowed by {book['student_name']} ({book['student_id_number']}) on {book['borrow_date']}, "
              f"was due on {book['due_date']}.")
else:
    print("No books are overdue as of today.")

# b. Call get_overdue_books with a past current_date
# Let's use a date from before Dune was returned (if it was) or just a historical date
historical_date = datetime.date.today() - datetime.timedelta(days=5)
print(f"\n--- Overdue books as of {historical_date} ---")
overdue_past = get_overdue_books(historical_date)
if overdue_past:
    for book in overdue_past:
        print(f"HISTORICAL REMINDER (as of {historical_date}): Book '{book['book_title']}' (ID: {book['book_id']}) by {book['book_author']} was overdue. "
              f"Borrowed by {book['student_name']} ({book['student_id_number']}) on {book['borrow_date']}, "
              f"was due on {book['due_date']}.")
else:
    print(f"No books were overdue as of {historical_date}.")



--- Testing Due-Date Reminders ---
Updated borrow_id 2 to have due_date: 2025-11-22 for testing.

--- Overdue books as of today ---
REMINDER: Book 'Dune' (ID: 5) by Frank Herbert is overdue. Borrowed by Student Two (S005) on 2025-12-02, was due on 2025-11-22.

--- Overdue books as of 2025-11-27 ---
HISTORICAL REMINDER (as of 2025-11-27): Book 'Dune' (ID: 5) by Frank Herbert was overdue. Borrowed by Student Two (S005) on 2025-12-02, was due on 2025-11-22.


## Develop Command-Line Interface (CLI)

### Subtask:
Create a simple command-line interface to interact with the implemented functionalities (Book CRUD, Student CRUD, Issue/Return, Reminders, Student Login).


## Develop Command-Line Interface (CLI)

### Subtask:
Create a simple command-line interface to interact with the implemented functionalities (Book CRUD, Student CRUD, Issue/Return, Reminders, Student Login).

**Reasoning**:
I will design a `main_menu` function to serve as the entry point for the CLI. This function will manage the user's session, handling login/logout, and displaying different menus based on the authenticated user's role (admin or student). I will create helper functions for admin and student specific menus (`admin_menu`, `student_menu`) to keep the code modular. User input will be processed, and appropriate functions from previous subtasks will be called to perform operations.


**Reasoning**:
I will implement the core command-line interface logic, including the main menu, login system, role-based menus (admin, student, guest), and integration of all previously defined database interaction functions.



In [ ]:
import sqlite3
import datetime
import os

# --- Helper function to clear screen ---
def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

# --- Helper function to get a database connection and cursor ---
def get_db_connection():
    conn = sqlite3.connect('library.db')
    conn.row_factory = sqlite3.Row # Allows accessing columns by name
    return conn

# --- Book CRUD Operations (re-included for self-contained CLI) ---
def add_book(title, author, isbn, quantity, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Books (title, author, isbn, quantity) VALUES (?, ?, ?, ?)",
                       (title, author, isbn, quantity))
        conn.commit()
        print(f"Book '{title}' added successfully.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A book with ISBN '{isbn}' already exists.")
        return False
    finally:
        if close_conn: conn.close()

def get_book(identifier, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT * FROM Books WHERE book_id = ? OR isbn = ?"
    cursor.execute(query, (identifier, identifier))
    book = cursor.fetchone()
    if close_conn: conn.close()
    if book:
        return {
            "book_id": book['book_id'],
            "title": book['title'],
            "author": book['author'],
            "isbn": book['isbn'],
            "quantity": book['quantity']
        }
    return None

def list_all_books(conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Books")
    books = cursor.fetchall()
    if close_conn: conn.close()
    book_list = []
    for book in books:
        book_list.append({
            "book_id": book['book_id'],
            "title": book['title'],
            "author": book['author'],
            "isbn": book['isbn'],
            "quantity": book['quantity']
        })
    return book_list

def update_book(book_id, title=None, author=None, isbn=None, quantity=None, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    updates = []
    params = []

    if title is not None:
        updates.append("title = ?")
        params.append(title)
    if author is not None:
        updates.append("author = ?")
        params.append(author)
    if isbn is not None:
        updates.append("isbn = ?")
        params.append(isbn)
    if quantity is not None:
        updates.append("quantity = ?")
        params.append(quantity)

    if not updates:
        if close_conn: conn.close()
        return False

    query = f"UPDATE Books SET {', '.join(updates)} WHERE book_id = ?"
    params.append(book_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            return True
        else:
            return False
    except sqlite3.IntegrityError:
        return False
    finally:
        if close_conn: conn.close()

def delete_book(book_id, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Books WHERE book_id = ?", (book_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Book with ID {book_id} deleted successfully.")
            return True
        else:
            print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot delete book with ID {book_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        if close_conn: conn.close()

# --- Student CRUD Operations (re-included for self-contained CLI) ---
def add_student(name, student_id_number, password, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Students (name, student_id_number, password) VALUES (?, ?, ?)",
                       (name, student_id_number, password))
        conn.commit()
        print(f"Student '{name}' added successfully with ID: {cursor.lastrowid}.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A student with ID number '{student_id_number}' already exists.")
        return False
    finally:
        if close_conn: conn.close()

def get_student(identifier, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id = ? OR student_id_number = ?"
    cursor.execute(query, (identifier, identifier))
    student = cursor.fetchone()
    if close_conn: conn.close()
    if student:
        return {
            "student_id": student['student_id'],
            "name": student['name'],
            "student_id_number": student['student_id_number']
        }
    return None

def list_all_students(conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    cursor.execute("SELECT student_id, name, student_id_number FROM Students")
    students = cursor.fetchall()
    if close_conn: conn.close()
    student_list = []
    for student in students:
        student_list.append({
            "student_id": student['student_id'],
            "name": student['name'],
            "student_id_number": student['student_id_number']
        })
    return student_list

def update_student(student_id, name=None, student_id_number=None, password=None, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    updates = []
    params = []

    if name is not None:
        updates.append("name = ?")
        params.append(name)
    if student_id_number is not None:
        updates.append("student_id_number = ?")
        params.append(student_id_number)
    if password is not None:
        updates.append("password = ?")
        params.append(password)

    if not updates:
        print("No update parameters provided.")
        if close_conn: conn.close()
        return False

    query = f"UPDATE Students SET {', '.join(updates)} WHERE student_id = ?"
    params.append(student_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} updated successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot update student. Student ID number '{student_id_number}' might already exist for another student.")
        return False
    finally:
        if close_conn: conn.close()

def delete_student(student_id, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Students WHERE student_id = ?", (student_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} deleted successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot delete student with ID {student_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        if close_conn: conn.close()

def login_student(student_id_number, password, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id_number = ? AND password = ?"
    cursor.execute(query, (student_id_number, password))
    student = cursor.fetchone()
    if close_conn: conn.close()
    if student:
        print(f"Login successful for {student['name']}.")
        return {"student_id": student['student_id'], "name": student['name'], "student_id_number": student['student_id_number']}
    else:
        print("Invalid student ID number or password.")
        return None

# --- Borrowing Operations (re-included for self-contained CLI) ---
def issue_book(book_id, student_id, borrow_date, due_date):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        # Check if book exists and is available
        book = get_book(book_id, conn) # Pass connection to helper
        if not book:
            print(f"Error: Book with ID {book_id} not found.")
            return False
        if book['quantity'] <= 0:
            print(f"Error: Book '{book['title']}' (ID: {book_id}) is currently out of stock.")
            return False

        # Check if student exists
        student = get_student(student_id, conn) # Pass connection to helper
        if not student:
            print(f"Error: Student with ID {student_id} not found.")
            return False

        # Insert into BorrowedBooks
        cursor.execute(
            "INSERT INTO BorrowedBooks (book_id, student_id, borrow_date, due_date) VALUES (?, ?, ?, ?)",
            (book_id, student_id, borrow_date, due_date)
        )
        borrow_id = cursor.lastrowid

        # Decrement book quantity
        update_book(book_id, quantity=book['quantity'] - 1, conn=conn) # Pass connection to helper

        conn.commit()
        print(f"Book '{book['title']}' issued to '{student['name']}' (Borrow ID: {borrow_id}) successfully.")
        return borrow_id
    except Exception as e:
        conn.rollback()
        print(f"Error issuing book: {e}")
        return False
    finally:
        conn.close()

def return_book(borrow_id, return_date):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        # Check if borrow record exists and is not already returned
        cursor.execute("SELECT book_id, return_date FROM BorrowedBooks WHERE borrow_id = ?", (borrow_id,))
        borrow_record = cursor.fetchone()

        if not borrow_record:
            print(f"Error: Borrow record with ID {borrow_id} not found.")
            return False
        if borrow_record['return_date'] is not None:
            print(f"Error: Book from borrow record ID {borrow_id} has already been returned on {borrow_record['return_date']}.")
            return False

        book_id = borrow_record['book_id']

        # Update return_date in BorrowedBooks
        cursor.execute("UPDATE BorrowedBooks SET return_date = ? WHERE borrow_id = ?", (return_date, borrow_id))

        # Increment book quantity
        book = get_book(book_id, conn) # Pass connection to helper
        if book:
            update_book(book_id, quantity=book['quantity'] + 1, conn=conn) # Pass connection to helper
        else:
            print(f"Warning: Book with ID {book_id} not found for incrementing quantity after return.")

        conn.commit()
        print(f"Book from borrow ID {borrow_id} returned successfully.")
        return True
    except Exception as e:
        conn.rollback()
        print(f"Error returning book: {e}")
        return False
    finally:
        conn.close()

# --- Overdue Reminders (re-included for self-contained CLI) ---
def get_overdue_books(current_date=None):
    conn = get_db_connection()
    cursor = conn.cursor()

    if current_date is None:
        current_date = datetime.date.today()
    elif isinstance(current_date, str):
        current_date = datetime.date.fromisoformat(current_date)

    # Convert current_date to string for SQL comparison
    current_date_str = str(current_date)

    query = """
    SELECT
        bb.borrow_id,
        b.book_id,
        b.title AS book_title,
        b.author AS book_author,
        s.student_id,
        s.name AS student_name,
        s.student_id_number,
        bb.borrow_date,
        bb.due_date
    FROM BorrowedBooks bb
    JOIN Books b ON bb.book_id = b.book_id
    JOIN Students s ON bb.student_id = s.student_id
    WHERE bb.return_date IS NULL AND bb.due_date < ?
    """
    cursor.execute(query, (current_date_str,))
    overdue_records = cursor.fetchall()
    conn.close()

    overdue_list = []
    for record in overdue_records:
        overdue_list.append(dict(record))
    return overdue_list

# --- CLI Functions ---
def display_book_details(book):
    if book:
        print(f"  ID: {book['book_id']}")
        print(f"  Title: {book['title']}")
        print(f"  Author: {book['author']}")
        print(f"  ISBN: {book['isbn']}")
        print(f"  Quantity: {book['quantity']}")
    else:
        print("Book not found.")

def display_student_details(student):
    if student:
        print(f"  ID: {student['student_id']}")
        print(f"  Name: {student['name']}")
        print(f"  Student ID Number: {student['student_id_number']}")
    else:
        print("Student not found.")

def display_borrowed_books(student_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    query = """
    SELECT
        bb.borrow_id, b.title, b.author, bb.borrow_date, bb.due_date, bb.return_date
    FROM BorrowedBooks bb
    JOIN Books b ON bb.book_id = b.book_id
    WHERE bb.student_id = ?
    ORDER BY bb.due_date DESC
    """
    cursor.execute(query, (student_id,))
    records = cursor.fetchall()
    conn.close()

    if not records:
        print("You have not borrowed any books.")
        return

    print("\n--- Your Borrowed Books ---")
    for record in records:
        print(f"Borrow ID: {record['borrow_id']}")
        print(f"  Title: {record['title']}")
        print(f"  Author: {record['author']}")
        print(f"  Borrowed On: {record['borrow_date']}")
        print(f"  Due On: {record['due_date']}")
        print(f"  Returned On: {record['return_date'] if record['return_date'] else 'Not Returned'}")
        print("-------------------------")

# --- Admin Menu Functions ---
def admin_book_menu():
    while True:
        print("\n--- Admin Book Management ---")
        print("1. Add New Book")
        print("2. List All Books")
        print("3. Get Book by ID/ISBN")
        print("4. Update Book Details")
        print("5. Delete Book")
        print("0. Back to Admin Main Menu")

        choice = input("Enter your choice: ")

        if choice == '1':
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            isbn = input("Enter book ISBN: ")
            while True:
                try:
                    quantity = int(input("Enter quantity: "))
                    if quantity < 0: raise ValueError
                    break
                except ValueError:
                    print("Invalid quantity. Please enter a non-negative integer.")
            add_book(title, author, isbn, quantity)
        elif choice == '2':
            books = list_all_books()
            if books:
                print("\n--- All Books ---")
                for book in books:
                    display_book_details(book)
                    print("-------------------------")
            else:
                print("No books in the library.")
        elif choice == '3':
            identifier = input("Enter Book ID or ISBN: ")
            try:
                identifier = int(identifier) # Try converting to int for ID
            except ValueError:
                pass # Keep as string for ISBN
            book = get_book(identifier)
            display_book_details(book)
        elif choice == '4':
            try:
                book_id = int(input("Enter the ID of the book to update: "))
                current_book = get_book(book_id)
                if not current_book:
                    print(f"No book found with ID {book_id}.")
                    continue

                print(f"Updating Book: {current_book['title']} (ID: {book_id})")
                title = input(f"Enter new title (current: {current_book['title']}) or leave blank: ") or None
                author = input(f"Enter new author (current: {current_book['author']}) or leave blank: ") or None
                isbn = input(f"Enter new ISBN (current: {current_book['isbn']}) or leave blank: ") or None
                quantity_str = input(f"Enter new quantity (current: {current_book['quantity']}) or leave blank: ")
                quantity = int(quantity_str) if quantity_str else None

                update_book(book_id, title, author, isbn, quantity)
            except ValueError:
                print("Invalid book ID. Please enter an integer.")
        elif choice == '5':
            try:
                book_id = int(input("Enter the ID of the book to delete: "))
                delete_book(book_id)
            except ValueError:
                print("Invalid book ID. Please enter an integer.")
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

def admin_student_menu():
    while True:
        print("\n--- Admin Student Management ---")
        print("1. Add New Student")
        print("2. List All Students")
        print("3. Get Student by ID/ID Number")
        print("4. Update Student Details")
        print("5. Delete Student")
        print("0. Back to Admin Main Menu")

        choice = input("Enter your choice: ")

        if choice == '1':
            name = input("Enter student name: ")
            student_id_num = input("Enter student ID number: ")
            password = input("Enter password: ")
            add_student(name, student_id_num, password)
        elif choice == '2':
            students = list_all_students()
            if students:
                print("\n--- All Students ---")
                for student in students:
                    display_student_details(student)
                    print("-------------------------")
            else:
                print("No students registered.")
        elif choice == '3':
            identifier = input("Enter Student ID or ID Number: ")
            try:
                identifier = int(identifier) # Try converting to int for ID
            except ValueError:
                pass # Keep as string for ID number
            student = get_student(identifier)
            display_student_details(student)
        elif choice == '4':
            try:
                student_id = int(input("Enter the ID of the student to update: "))
                current_student = get_student(student_id)
                if not current_student:
                    print(f"No student found with ID {student_id}.")
                    continue

                print(f"Updating Student: {current_student['name']} (ID: {student_id})")
                name = input(f"Enter new name (current: {current_student['name']}) or leave blank: ") or None
                student_id_number = input(f"Enter new student ID number (current: {current_student['student_id_number']}) or leave blank: ") or None
                password = input("Enter new password (leave blank to keep current): ") or None

                update_student(student_id, name, student_id_number, password)
            except ValueError:
                print("Invalid student ID. Please enter an integer.")
        elif choice == '5':
            try:
                student_id = int(input("Enter the ID of the student to delete: "))
                delete_student(student_id)
            except ValueError:
                print("Invalid student ID. Please enter an integer.")
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

def admin_issue_return_menu():
    while True:
        print("\n--- Admin Issue/Return Management ---")
        print("1. Issue Book")
        print("2. Return Book")
        print("0. Back to Admin Main Menu")

        choice = input("Enter your choice: ")

        if choice == '1':
            try:
                book_id = int(input("Enter Book ID: "))
                student_id = int(input("Enter Student ID: "))
                borrow_date = str(datetime.date.today())
                due_date_days = input("Enter due date in days from today (e.g., 7 for one week): ")
                due_date = str(datetime.date.today() + datetime.timedelta(days=int(due_date_days)))
                issue_book(book_id, student_id, borrow_date, due_date)
            except ValueError:
                print("Invalid ID or days. Please enter integers.")
        elif choice == '2':
            try:
                borrow_id = int(input("Enter Borrow ID to return: "))
                return_date = str(datetime.date.today())
                return_book(borrow_id, return_date)
            except ValueError:
                print("Invalid Borrow ID. Please enter an integer.")
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

def admin_reminders_menu():
    print("\n--- Overdue Book Reminders ---")
    overdue_books = get_overdue_books()
    if overdue_books:
        for book in overdue_books:
            print(f"REMINDER: Book '{book['book_title']}' (ID: {book['book_id']}) by {book['book_author']} is overdue. ")
            print(f"  Borrowed by {book['student_name']} ({book['student_id_number']}) on {book['borrow_date']}, ")
            print(f"  Was due on {book['due_date']}.")
            print("-------------------------")
    else:
        print("No books are currently overdue.")
    input("Press Enter to continue...")

def admin_menu():
    while True:
        clear_screen()
        print("\n--- Admin Main Menu ---")
        print("1. Book Management")
        print("2. Student Management")
        print("3. Issue/Return Books")
        print("4. Generate Overdue Reminders")
        print("0. Logout")

        choice = input("Enter your choice: ")

        if choice == '1':
            admin_book_menu()
        elif choice == '2':
            admin_student_menu()
        elif choice == '3':
            admin_issue_return_menu()
        elif choice == '4':
            admin_reminders_menu()
        elif choice == '0':
            print("Logging out from Admin account.")
            return
        else:
            print("Invalid choice. Please try again.")
            input("Press Enter to continue...")

# --- Student Menu Functions ---
def student_view_all_books():
    books = list_all_books()
    if books:
        print("\n--- Available Books ---")
        for book in books:
            print(f"ID: {book['book_id']}, Title: {book['title']}, Author: {book['author']}, Quantity: {book['quantity']}")
        print("-------------------------")
    else:
        print("No books currently available in the library.")
    input("Press Enter to continue...")

def student_return_book(student_id):
    display_borrowed_books(student_id)
    if input("Do you want to return a book? (y/n): ").lower() == 'y':
        try:
            borrow_id = int(input("Enter the Borrow ID of the book you want to return: "))
            return_date = str(datetime.date.today())
            return_book(borrow_id, return_date)
        except ValueError:
            print("Invalid Borrow ID. Please enter an integer.")
    input("Press Enter to continue...")

def student_menu(logged_in_student):
    while True:
        clear_screen()
        print(f"\n--- Welcome, {logged_in_student['name']} (Student ID: {logged_in_student['student_id_number']}) ---")
        print("1. View All Available Books")
        print("2. View Your Borrowed Books")
        print("3. Return a Book")
        print("0. Logout")

        choice = input("Enter your choice: ")

        if choice == '1':
            student_view_all_books()
        elif choice == '2':
            display_borrowed_books(logged_in_student['student_id'])
            input("Press Enter to continue...")
        elif choice == '3':
            student_return_book(logged_in_student['student_id'])
        elif choice == '0':
            print("Logging out.")
            return
        else:
            print("Invalid choice. Please try again.")
            input("Press Enter to continue...")

# --- Main CLI Loop ---
def main_menu():
    logged_in_user = None # Stores student info if logged in
    is_admin = False

    # Admin credentials (for simplicity, hardcoded)
    ADMIN_ID = "admin"
    ADMIN_PASS = "admin123"

    while True:
        clear_screen()
        print("\n--- Library Management System CLI ---")
        if logged_in_user:
            if is_admin:
                print(f"Logged in as Admin.")
                admin_menu()
            else:
                print(f"Logged in as Student: {logged_in_user['name']}")
                student_menu(logged_in_user)
            logged_in_user = None # Logout after menu exits
            is_admin = False
        else:
            print("1. Student Login")
            print("2. Admin Login")
            print("3. Register New Student")
            print("0. Exit Application")

            choice = input("Enter your choice: ")

            if choice == '1':
                student_id_number = input("Enter your student ID number: ")
                password = input("Enter your password: ")
                student = login_student(student_id_number, password)
                if student:
                    logged_in_user = student
                    is_admin = False
                input("Press Enter to continue...")
            elif choice == '2':
                username = input("Enter admin username: ")
                password = input("Enter admin password: ")
                if username == ADMIN_ID and password == ADMIN_PASS:
                    print("Admin login successful!")
                    logged_in_user = {"name": "Admin", "student_id": 0, "student_id_number": ADMIN_ID} # Dummy user object
                    is_admin = True
                else:
                    print("Invalid admin credentials.")
                input("Press Enter to continue...")
            elif choice == '3':
                name = input("Enter your full name: ")
                student_id_number = input("Choose a student ID number: ")
                password = input("Choose a password: ")
                add_student(name, student_id_number, password)
                input("Press Enter to continue...")
            elif choice == '0':
                print("Exiting Library Management System. Goodbye!")
                break
            else:
                print("Invalid choice. Please try again.")
                input("Press Enter to continue...")

# --- Initialize and run the CLI ---
if __name__ == '__main__':
    # Ensure database is initialized before starting CLI
    # (This part would ideally be in a separate setup script or main entry)
    # For this exercise, assuming library.db and tables exist from earlier cells.
    print("Starting Library Management System CLI...")
    # You might want to uncomment and run initialize_database() if it were a function
    # to ensure tables are created, but it was a direct script in the first cell.
    main_menu()

Starting Library Management System CLI...

--- Library Management System CLI ---
1. Student Login
2. Admin Login
3. Register New Student
0. Exit Application
Enter your choice: 0
Exiting Library Management System. Goodbye!


**Reasoning**:
The previous code failed due to a `SyntaxError` on line 120: `cursor = conn.cursor()l`. There was an extraneous 'l' character after `conn.cursor()`. I will correct this syntax error by removing the 'l'.



In [ ]:
import sqlite3
import datetime
import os

# --- Helper function to clear screen ---
def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

# --- Helper function to get a database connection and cursor ---
def get_db_connection():
    conn = sqlite3.connect('library.db')
    conn.row_factory = sqlite3.Row # Allows accessing columns by name
    return conn

# --- Book CRUD Operations (re-included for self-contained CLI) ---
def add_book(title, author, isbn, quantity, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Books (title, author, isbn, quantity) VALUES (?, ?, ?, ?)",
                       (title, author, isbn, quantity))
        conn.commit()
        print(f"Book '{title}' added successfully.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A book with ISBN '{isbn}' already exists.")
        return False
    finally:
        if close_conn: conn.close()

def get_book(identifier, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT * FROM Books WHERE book_id = ? OR isbn = ?"
    cursor.execute(query, (identifier, identifier))
    book = cursor.fetchone()
    if close_conn: conn.close()
    if book:
        return {
            "book_id": book['book_id'],
            "title": book['title'],
            "author": book['author'],
            "isbn": book['isbn'],
            "quantity": book['quantity']
        }
    return None

def list_all_books(conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Books")
    books = cursor.fetchall()
    if close_conn: conn.close()
    book_list = []
    for book in books:
        book_list.append({
            "book_id": book['book_id'],
            "title": book['title'],
            "author": book['author'],
            "isbn": book['isbn'],
            "quantity": book['quantity']
        })
    return book_list

def update_book(book_id, title=None, author=None, isbn=None, quantity=None, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    updates = []
    params = []

    if title is not None:
        updates.append("title = ?")
        params.append(title)
    if author is not None:
        updates.append("author = ?")
        params.append(author)
    if isbn is not None:
        updates.append("isbn = ?")
        params.append(isbn)
    if quantity is not None:
        updates.append("quantity = ?")
        params.append(quantity)

    if not updates:
        if close_conn: conn.close()
        return False

    query = f"UPDATE Books SET {', '.join(updates)} WHERE book_id = ?"
    params.append(book_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            return True
        else:
            return False
    except sqlite3.IntegrityError:
        return False
    finally:
        if close_conn: conn.close()

def delete_book(book_id, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Books WHERE book_id = ?", (book_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Book with ID {book_id} deleted successfully.")
            return True
        else:
            print(f"No book found with ID {book_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot delete book with ID {book_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        if close_conn: conn.close()

# --- Student CRUD Operations (re-included for self-contained CLI) ---
def add_student(name, student_id_number, password, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Students (name, student_id_number, password) VALUES (?, ?, ?)",
                       (name, student_id_number, password))
        conn.commit()
        print(f"Student '{name}' added successfully with ID: {cursor.lastrowid}.")
        return True
    except sqlite3.IntegrityError:
        print(f"Error: A student with ID number '{student_id_number}' already exists.")
        return False
    finally:
        if close_conn: conn.close()

def get_student(identifier, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id = ? OR student_id_number = ?"
    cursor.execute(query, (identifier, identifier))
    student = cursor.fetchone()
    if close_conn: conn.close()
    if student:
        return {
            "student_id": student['student_id'],
            "name": student['name'],
            "student_id_number": student['student_id_number']
        }
    return None

def list_all_students(conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    cursor.execute("SELECT student_id, name, student_id_number FROM Students")
    students = cursor.fetchall()
    if close_conn: conn.close()
    student_list = []
    for student in students:
        student_list.append({
            "student_id": student['student_id'],
            "name": student['name'],
            "student_id_number": student['student_id_number']
        })
    return student_list

def update_student(student_id, name=None, student_id_number=None, password=None, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    updates = []
    params = []

    if name is not None:
        updates.append("name = ?")
        params.append(name)
    if student_id_number is not None:
        updates.append("student_id_number = ?")
        params.append(student_id_number)
    if password is not None:
        updates.append("password = ?")
        params.append(password)

    if not updates:
        print("No update parameters provided.")
        if close_conn: conn.close()
        return False

    query = f"UPDATE Students SET {', '.join(updates)} WHERE student_id = ?"
    params.append(student_id)

    try:
        cursor.execute(query, tuple(params))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} updated successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot update student. Student ID number '{student_id_number}' might already exist for another student.")
        return False
    finally:
        if close_conn: conn.close()

def delete_student(student_id, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    try:
        cursor.execute("DELETE FROM Students WHERE student_id = ?", (student_id,))
        conn.commit()
        if cursor.rowcount > 0:
            print(f"Student with ID {student_id} deleted successfully.")
            return True
        else:
            print(f"No student found with ID {student_id}.")
            return False
    except sqlite3.IntegrityError:
        print(f"Error: Cannot delete student with ID {student_id}. It might be referenced in BorrowedBooks table.")
        return False
    finally:
        if close_conn: conn.close()

def login_student(student_id_number, password, conn=None):
    close_conn = False
    if conn is None:
        conn = get_db_connection()
        close_conn = True
    cursor = conn.cursor()
    query = "SELECT student_id, name, student_id_number FROM Students WHERE student_id_number = ? AND password = ?"
    cursor.execute(query, (student_id_number, password))
    student = cursor.fetchone()
    if close_conn: conn.close()
    if student:
        print(f"Login successful for {student['name']}.")
        return {"student_id": student['student_id'], "name": student['name'], "student_id_number": student['student_id_number']}
    else:
        print("Invalid student ID number or password.")
        return None

# --- Borrowing Operations (re-included for self-contained CLI) ---
def issue_book(book_id, student_id, borrow_date, due_date):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        # Check if book exists and is available
        book = get_book(book_id, conn) # Pass connection to helper
        if not book:
            print(f"Error: Book with ID {book_id} not found.")
            return False
        if book['quantity'] <= 0:
            print(f"Error: Book '{book['title']}' (ID: {book_id}) is currently out of stock.")
            return False

        # Check if student exists
        student = get_student(student_id, conn) # Pass connection to helper
        if not student:
            print(f"Error: Student with ID {student_id} not found.")
            return False

        # Insert into BorrowedBooks
        cursor.execute(
            "INSERT INTO BorrowedBooks (book_id, student_id, borrow_date, due_date) VALUES (?, ?, ?, ?)",
            (book_id, student_id, borrow_date, due_date)
        )
        borrow_id = cursor.lastrowid

        # Decrement book quantity
        update_book(book_id, quantity=book['quantity'] - 1, conn=conn) # Pass connection to helper

        conn.commit()
        print(f"Book '{book['title']}' issued to '{student['name']}' (Borrow ID: {borrow_id}) successfully.")
        return borrow_id
    except Exception as e:
        conn.rollback()
        print(f"Error issuing book: {e}")
        return False
    finally:
        conn.close()

def return_book(borrow_id, return_date):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        # Check if borrow record exists and is not already returned
        cursor.execute("SELECT book_id, return_date FROM BorrowedBooks WHERE borrow_id = ?", (borrow_id,))
        borrow_record = cursor.fetchone()

        if not borrow_record:
            print(f"Error: Borrow record with ID {borrow_id} not found.")
            return False
        if borrow_record['return_date'] is not None:
            print(f"Error: Book from borrow record ID {borrow_id} has already been returned on {borrow_record['return_date']}.")
            return False

        book_id = borrow_record['book_id']

        # Update return_date in BorrowedBooks
        cursor.execute("UPDATE BorrowedBooks SET return_date = ? WHERE borrow_id = ?", (return_date, borrow_id))

        # Increment book quantity
        book = get_book(book_id, conn) # Pass connection to helper
        if book:
            update_book(book_id, quantity=book['quantity'] + 1, conn=conn) # Pass connection to helper
        else:
            print(f"Warning: Book with ID {book_id} not found for incrementing quantity after return.")

        conn.commit()
        print(f"Book from borrow ID {borrow_id} returned successfully.")
        return True
    except Exception as e:
        conn.rollback()
        print(f"Error returning book: {e}")
        return False
    finally:
        conn.close()

# --- Overdue Reminders (re-included for self-contained CLI) ---
def get_overdue_books(current_date=None):
    conn = get_db_connection()
    cursor = conn.cursor()

    if current_date is None:
        current_date = datetime.date.today()
    elif isinstance(current_date, str):
        current_date = datetime.date.fromisoformat(current_date)

    # Convert current_date to string for SQL comparison
    current_date_str = str(current_date)

    query = """
    SELECT
        bb.borrow_id,
        b.book_id,
        b.title AS book_title,
        b.author AS book_author,
        s.student_id,
        s.name AS student_name,
        s.student_id_number,
        bb.borrow_date,
        bb.due_date
    FROM BorrowedBooks bb
    JOIN Books b ON bb.book_id = b.book_id
    JOIN Students s ON bb.student_id = s.student_id
    WHERE bb.return_date IS NULL AND bb.due_date < ?
    """
    cursor.execute(query, (current_date_str,))
    overdue_records = cursor.fetchall()
    conn.close()

    overdue_list = []
    for record in overdue_records:
        overdue_list.append(dict(record))
    return overdue_list

# --- CLI Functions ---
def display_book_details(book):
    if book:
        print(f"  ID: {book['book_id']}")
        print(f"  Title: {book['title']}")
        print(f"  Author: {book['author']}")
        print(f"  ISBN: {book['isbn']}")
        print(f"  Quantity: {book['quantity']}")
    else:
        print("Book not found.")

def display_student_details(student):
    if student:
        print(f"  ID: {student['student_id']}")
        print(f"  Name: {student['name']}")
        print(f"  Student ID Number: {student['student_id_number']}")
    else:
        print("Student not found.")

def display_borrowed_books(student_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    query = """
    SELECT
        bb.borrow_id, b.title, b.author, bb.borrow_date, bb.due_date, bb.return_date
    FROM BorrowedBooks bb
    JOIN Books b ON bb.book_id = b.book_id
    WHERE bb.student_id = ?
    ORDER BY bb.due_date DESC
    """
    cursor.execute(query, (student_id,))
    records = cursor.fetchall()
    conn.close()

    if not records:
        print("You have not borrowed any books.")
        return

    print("\n--- Your Borrowed Books ---")
    for record in records:
        print(f"Borrow ID: {record['borrow_id']}")
        print(f"  Title: {record['title']}")
        print(f"  Author: {record['author']}")
        print(f"  Borrowed On: {record['borrow_date']}")
        print(f"  Due On: {record['due_date']}")
        print(f"  Returned On: {record['return_date'] if record['return_date'] else 'Not Returned'}")
        print("-------------------------")

# --- Admin Menu Functions ---
def admin_book_menu():
    while True:
        print("\n--- Admin Book Management ---")
        print("1. Add New Book")
        print("2. List All Books")
        print("3. Get Book by ID/ISBN")
        print("4. Update Book Details")
        print("5. Delete Book")
        print("0. Back to Admin Main Menu")

        choice = input("Enter your choice: ")

        if choice == '1':
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            isbn = input("Enter book ISBN: ")
            while True:
                try:
                    quantity = int(input("Enter quantity: "))
                    if quantity < 0: raise ValueError
                    break
                except ValueError:
                    print("Invalid quantity. Please enter a non-negative integer.")
            add_book(title, author, isbn, quantity)
        elif choice == '2':
            books = list_all_books()
            if books:
                print("\n--- All Books ---")
                for book in books:
                    display_book_details(book)
                    print("-------------------------")
            else:
                print("No books in the library.")
        elif choice == '3':
            identifier = input("Enter Book ID or ISBN: ")
            try:
                identifier = int(identifier) # Try converting to int for ID
            except ValueError:
                pass # Keep as string for ISBN
            book = get_book(identifier)
            display_book_details(book)
        elif choice == '4':
            try:
                book_id = int(input("Enter the ID of the book to update: "))
                current_book = get_book(book_id)
                if not current_book:
                    print(f"No book found with ID {book_id}.")
                    continue

                print(f"Updating Book: {current_book['title']} (ID: {book_id})")
                title = input(f"Enter new title (current: {current_book['title']}) or leave blank: ") or None
                author = input(f"Enter new author (current: {current_book['author']}) or leave blank: ") or None
                isbn = input(f"Enter new ISBN (current: {current_book['isbn']}) or leave blank: ") or None
                quantity_str = input(f"Enter new quantity (current: {current_book['quantity']}) or leave blank: ")
                quantity = int(quantity_str) if quantity_str else None

                update_book(book_id, title, author, isbn, quantity)
            except ValueError:
                print("Invalid book ID. Please enter an integer.")
        elif choice == '5':
            try:
                book_id = int(input("Enter the ID of the book to delete: "))
                delete_book(book_id)
            except ValueError:
                print("Invalid book ID. Please enter an integer.")
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

def admin_student_menu():
    while True:
        print("\n--- Admin Student Management ---")
        print("1. Add New Student")
        print("2. List All Students")
        print("3. Get Student by ID/ID Number")
        print("4. Update Student Details")
        print("5. Delete Student")
        print("0. Back to Admin Main Menu")

        choice = input("Enter your choice: ")

        if choice == '1':
            name = input("Enter student name: ")
            student_id_num = input("Enter student ID number: ")
            password = input("Enter password: ")
            add_student(name, student_id_num, password)
        elif choice == '2':
            students = list_all_students()
            if students:
                print("\n--- All Students ---")
                for student in students:
                    display_student_details(student)
                    print("-------------------------")
            else:
                print("No students registered.")
        elif choice == '3':
            identifier = input("Enter Student ID or ID Number: ")
            try:
                identifier = int(identifier) # Try converting to int for ID
            except ValueError:
                pass # Keep as string for ID number
            student = get_student(identifier)
            display_student_details(student)
        elif choice == '4':
            try:
                student_id = int(input("Enter the ID of the student to update: "))
                current_student = get_student(student_id)
                if not current_student:
                    print(f"No student found with ID {student_id}.")
                    continue

                print(f"Updating Student: {current_student['name']} (ID: {student_id})")
                name = input(f"Enter new name (current: {current_student['name']}) or leave blank: ") or None
                student_id_number = input(f"Enter new student ID number (current: {current_student['student_id_number']}) or leave blank: ") or None
                password = input("Enter new password (leave blank to keep current): ") or None

                update_student(student_id, name, student_id_number, password)
            except ValueError:
                print("Invalid student ID. Please enter an integer.")
        elif choice == '5':
            try:
                student_id = int(input("Enter the ID of the student to delete: "))
                delete_student(student_id)
            except ValueError:
                print("Invalid student ID. Please enter an integer.")
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

def admin_issue_return_menu():
    while True:
        print("\n--- Admin Issue/Return Management ---")
        print("1. Issue Book")
        print("2. Return Book")
        print("0. Back to Admin Main Menu")

        choice = input("Enter your choice: ")

        if choice == '1':
            try:
                book_id = int(input("Enter Book ID: "))
                student_id = int(input("Enter Student ID: "))
                borrow_date = str(datetime.date.today())
                due_date_days = input("Enter due date in days from today (e.g., 7 for one week): ")
                due_date = str(datetime.date.today() + datetime.timedelta(days=int(due_date_days)))
                issue_book(book_id, student_id, borrow_date, due_date)
            except ValueError:
                print("Invalid ID or days. Please enter integers.")
        elif choice == '2':
            try:
                borrow_id = int(input("Enter Borrow ID to return: "))
                return_date = str(datetime.date.today())
                return_book(borrow_id, return_date)
            except ValueError:
                print("Invalid Borrow ID. Please enter an integer.")
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

def admin_reminders_menu():
    print("\n--- Overdue Book Reminders ---")
    overdue_books = get_overdue_books()
    if overdue_books:
        for book in overdue_books:
            print(f"REMINDER: Book '{book['book_title']}' (ID: {book['book_id']}) by {book['book_author']} is overdue. ")
            print(f"  Borrowed by {book['student_name']} ({book['student_id_number']}) on {book['borrow_date']}, ")
            print(f"  Was due on {book['due_date']}.")
            print("-------------------------")
    else:
        print("No books are currently overdue.")
    input("Press Enter to continue...")

def admin_menu():
    while True:
        clear_screen()
        print("\n--- Admin Main Menu ---")
        print("1. Book Management")
        print("2. Student Management")
        print("3. Issue/Return Books")
        print("4. Generate Overdue Reminders")
        print("0. Logout")

        choice = input("Enter your choice: ")

        if choice == '1':
            admin_book_menu()
        elif choice == '2':
            admin_student_menu()
        elif choice == '3':
            admin_issue_return_menu()
        elif choice == '4':
            admin_reminders_menu()
        elif choice == '0':
            print("Logging out from Admin account.")
            return
        else:
            print("Invalid choice. Please try again.")
            input("Press Enter to continue...")

# --- Student Menu Functions ---
def student_view_all_books():
    books = list_all_books()
    if books:
        print("\n--- Available Books ---")
        for book in books:
            print(f"ID: {book['book_id']}, Title: {book['title']}, Author: {book['author']}, Quantity: {book['quantity']}")
        print("-------------------------")
    else:
        print("No books currently available in the library.")
    input("Press Enter to continue...")

def student_return_book(student_id):
    display_borrowed_books(student_id)
    if input("Do you want to return a book? (y/n): ").lower() == 'y':
        try:
            borrow_id = int(input("Enter the Borrow ID of the book you want to return: "))
            return_date = str(datetime.date.today())
            return_book(borrow_id, return_date)
        except ValueError:
            print("Invalid Borrow ID. Please enter an integer.")
    input("Press Enter to continue...")

def student_menu(logged_in_student):
    while True:
        clear_screen()
        print(f"\n--- Welcome, {logged_in_student['name']} (Student ID: {logged_in_student['student_id_number']}) ---")
        print("1. View All Available Books")
        print("2. View Your Borrowed Books")
        print("3. Return a Book")
        print("0. Logout")

        choice = input("Enter your choice: ")

        if choice == '1':
            student_view_all_books()
        elif choice == '2':
            display_borrowed_books(logged_in_student['student_id'])
            input("Press Enter to continue...")
        elif choice == '3':
            student_return_book(logged_in_student['student_id'])
        elif choice == '0':
            print("Logging out.")
            return
        else:
            print("Invalid choice. Please try again.")
            input("Press Enter to continue...")

# --- Main CLI Loop ---
def main_menu():
    logged_in_user = None # Stores student info if logged in
    is_admin = False

    # Admin credentials (for simplicity, hardcoded)
    ADMIN_ID = "admin"
    ADMIN_PASS = "admin123"

    while True:
        clear_screen()
        print("\n--- Library Management System CLI ---")
        if logged_in_user:
            if is_admin:
                print(f"Logged in as Admin.")
                admin_menu()
            else:
                print(f"Logged in as Student: {logged_in_user['name']}")
                student_menu(logged_in_user)
            logged_in_user = None # Logout after menu exits
            is_admin = False
        else:
            print("1. Student Login")
            print("2. Admin Login")
            print("3. Register New Student")
            print("0. Exit Application")

            choice = input("Enter your choice: ")

            if choice == '1':
                student_id_number = input("Enter your student ID number: ")
                password = input("Enter your password: ")
                student = login_student(student_id_number, password)
                if student:
                    logged_in_user = student
                    is_admin = False
                input("Press Enter to continue...")
            elif choice == '2':
                username = input("Enter admin username: ")
                password = input("Enter admin password: ")
                if username == ADMIN_ID and password == ADMIN_PASS:
                    print("Admin login successful!")
                    logged_in_user = {"name": "Admin", "student_id": 0, "student_id_number": ADMIN_ID} # Dummy user object
                    is_admin = True
                else:
                    print("Invalid admin credentials.")
                input("Press Enter to continue...")
            elif choice == '3':
                name = input("Enter your full name: ")
                student_id_number = input("Choose a student ID number: ")
                password = input("Choose a password: ")
                add_student(name, student_id_number, password)
                input("Press Enter to continue...")
            elif choice == '0':
                print("Exiting Library Management System. Goodbye!")
                break
            else:
                print("Invalid choice. Please try again.")
                input("Press Enter to continue...")

# --- Initialize and run the CLI ---
if __name__ == '__main__':
    # Ensure database is initialized before starting CLI
    # (This part would ideally be in a separate setup script or main entry)
    # For this exercise, assuming library.db and tables exist from earlier cells.
    print("Starting Library Management System CLI...")
    # You might want to uncomment and run initialize_database() if it were a function
    # to ensure tables are created, but it was a direct script in the first cell.
    main_menu()


Starting Library Management System CLI...

--- Library Management System CLI ---
1. Student Login
2. Admin Login
3. Register New Student
0. Exit Application
Enter your choice: 1
Enter your student ID number: 23
Enter your password: 2334456
Invalid student ID number or password.
Press Enter to continue...

--- Library Management System CLI ---
1. Student Login
2. Admin Login
3. Register New Student
0. Exit Application
Enter your choice: 0
Exiting Library Management System. Goodbye!


## Final Task

### Subtask:
Summarize the implemented Library Management System features and discuss next steps for expanding to a GUI or web version.


## Summary:

### Data Analysis Key Findings

*   **Database Foundation**: The `library.db` database was successfully initialized with three core tables: `Books`, `Students`, and `BorrowedBooks`. These tables are structured with appropriate primary and foreign keys, as well as unique constraints for `ISBN` (Books) and `student_id_number` (Students), ensuring data integrity.
*   **Comprehensive CRUD Operations**: Full Create, Read, Update, and Delete (CRUD) functionalities were implemented and verified for both `Books` and `Students`. This includes robust error handling for duplicate entries (e.g., duplicate ISBNs or student ID numbers) and non-existent records.
*   **Student Authentication**: A basic student login system was integrated, allowing users to authenticate with their `student_id_number` and `password`.
*   **Transaction Management for Borrowing**: The `issue_book` and `return_book` functionalities were successfully implemented, managing the `BorrowedBooks` table and dynamically updating book quantities. A significant challenge with concurrent database access ("database is locked" error) was resolved by refactoring database connection handling to ensure a single connection per transaction.
*   **Overdue Reminders**: A function `get_overdue_books` was developed to identify and retrieve details for books that are past their `due_date` and have not yet been returned. This function supports querying for both current and historical overdue statuses.
*   **Integrated Command-Line Interface (CLI)**: A functional command-line interface was developed, unifying all implemented features. It provides distinct menus for admin and student roles, handles user input, and interacts seamlessly with the underlying database functions for book management, student management, book issuance/return, and overdue reminders.

### Insights or Next Steps

*   The current command-line interface (CLI) provides a solid functional prototype. The natural next step would be to expand this system into a graphical user interface (GUI) using libraries like Tkinter or PyQt for a desktop application, or a web application using frameworks like Flask or Django for broader accessibility.
*   For the current CLI, adding more sophisticated input validation (e.g., date formats, string length checks) and robust error messages for user-facing interactions would enhance user experience and system reliability.
